In [1]:
import pandas as pd

# Beer League Positionator 2000

Because the 1999 version sucked.

The idea is to completely over-engineer the simple idea of creating lines on a beer league hockey team.  If this lasts a few days, I'll have a Flask app for data entry talking via ZeroMQ to Chapel backend to perform the optimization.  In short

1. Each player takes a survey on preferences.
1. When available, game data is entered into the BLP2000 concerning who was on the ice when a goal was scored or lost.
1. The BLP2000 optimizes the lines given the currently available players.

## Survey

Each play answers a survey for each position on the team with the scale below.  Repeated answers are fine (you can have as mny 2's or 5's or 0's as you want).

* 0 - Do not put me in this position. Really.
* 1 - I don't think I'd know what to do in this position.  We'll lose badly.
* 2 - I've seen videos and I think I know what to do here, I wouldn't mind trying it.
* 3 - I've played this position in scrimmage and I kinda like it.
* 4 - I've played this position in games and I can compete in a beer league.
* 5 - I'm beer league competitive in this position.

Each row has player preferences, and this would construct the $n-$player by 6 matrix $W$.  Here is an example of $W$.

In [6]:
w = pd.read_csv('data/players.txt')
w

,player,chirality,g,lw,ld,c,rd,rw
0,brian,L,0,4,3,2,1,1
1,alice,L,0,4,3,2,1,1
2,chuck,L,0,4,3,2,1,1
3,dani,L,0,4,3,2,1,1
4,efram,L,0,4,3,2,1,1
5,fionn,L,0,4,3,2,1,1
6,gabe,L,0,4,3,2,1,1
7,hilda,L,0,4,3,2,1,1
8,iona,L,0,4,3,2,1,1
9,jack,L,0,4,3,2,1,1


## Method

Let $W$ be the matrix of weights offered by players.  

$A$ is a 6$\times$14 (13 skaters + Goalie) random matrix assigning players to positions. The values $x=A(brian)$ is the assignemnt of Brian to position $x$.

$R$ is the penalty matrix for the assigned positions. This is determined as $max_{W_{brian}}\{W_{brian,j}\} - A_{brian,j}$ with $j$ ranging along the position columns of $W$. That is, if Brian states he is a 1 for _RW_ and he get assigned _RW_, then 

$$
\begin{eqnarray}
R_{brian} &=& max_{brian,}\{W_{brian,j}\}-A_{brian,j}\\
  &=& 4-1 \\
  &=& 3
\end{eqnarray}
$$

Here is an example $A$. Actually, this is the transpose because that's easier to read.  This assigns the first player to Goal (in this case Brian) and the next three to Left Wing, the next three to Left D, etc.

In [9]:
sample_a = pd.read_csv('data/sample_a.txt')
sample_a

,g,lw,ld,c,rd,rw
0,1,0,0,0,0,0
1,0,1,0,0,0,0
2,0,1,0,0,0,0
3,0,1,0,0,0,0
4,0,0,1,0,0,0
5,0,0,1,0,0,0
6,0,0,1,0,0,0
7,0,0,0,1,0,0
8,0,0,0,1,0,0
9,0,0,0,1,0,0
